<a href="https://colab.research.google.com/github/AbhimanyuAryan/uc/blob/master/uc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
!pip install git+https://github.com/fastai/fastai@2e1ccb58121dc648751e2109fc0fbf6925aa8887
!apt update && apt install -y libsm6 libxext6

In [0]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas_summary import DataFrameSummary
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from time import time
from fastai.imports import *
from fastai.structured import *

In [0]:
df_raw = pd.read_csv('data.csv', low_memory=False, 
                     parse_dates=["event_time"])

In [0]:
# Data Dec 17,2018 to Jan 30,2019
# Train Model on 17th to Jan 13

# predict values for Jan 14 - Jan 20
# what happends to data from 20th - 30th

In [32]:
# split data 57% - train 
# Rest is test data

df_raw.shape
# 57 % would be -  465423

(816530, 8)

In [0]:
df_raw['req_id'].fillna(0, inplace=True)

In [33]:
df_raw.head()

,record_id,city_key,event_time,weekday,category_key,rptcatg,req_id,session_group
0,5c1699b0c3658c25008e1192,city_chennai_v2,2018-12-17 00:00:08,weekday,professional_bathroom_cleaning,Cleaning - Other,1,3022
1,5c1699b72dd2b92500f5f400,city_mumbai_v2,2018-12-17 00:00:15,weekday,electricians,EPC,1,3303
2,5c1699df4fb96a2400166132,city_mumbai_v2,2018-12-17 00:00:55,weekday,salon_at_home,Salon at Home,0,3330
3,5c1699f405cbce26002ff832,city_kolkata_v2,2018-12-17 00:01:16,weekday,salon_at_home,Salon at Home,0,3330
4,5c1699fb02f88224006901f5,city_chennai_v2,2018-12-17 00:01:23,weekday,plumbers,EPC,1,3303


In [0]:
df_raw.loc[df_raw['req_id'] != 0, 'req_id'] = 1

In [35]:
df_raw.head()

,record_id,city_key,event_time,weekday,category_key,rptcatg,req_id,session_group
0,5c1699b0c3658c25008e1192,city_chennai_v2,2018-12-17 00:00:08,weekday,professional_bathroom_cleaning,Cleaning - Other,1,3022
1,5c1699b72dd2b92500f5f400,city_mumbai_v2,2018-12-17 00:00:15,weekday,electricians,EPC,1,3303
2,5c1699df4fb96a2400166132,city_mumbai_v2,2018-12-17 00:00:55,weekday,salon_at_home,Salon at Home,0,3330
3,5c1699f405cbce26002ff832,city_kolkata_v2,2018-12-17 00:01:16,weekday,salon_at_home,Salon at Home,0,3330
4,5c1699fb02f88224006901f5,city_chennai_v2,2018-12-17 00:01:23,weekday,plumbers,EPC,1,3303


In [0]:
# making more columns from date
add_datepart(df_raw, 'event_time')

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call train_cats to convert strings to pandas categories.

In [0]:
train_cats(df_raw)

In [41]:
df_raw.head()

,record_id,city_key,weekday,category_key,rptcatg,req_id,session_group,event_timeYear,event_timeMonth,event_timeWeek,event_timeDay,event_timeDayofweek,event_timeDayofyear,event_timeIs_month_end,event_timeIs_month_start,event_timeIs_quarter_end,event_timeIs_quarter_start,event_timeIs_year_end,event_timeIs_year_start,event_timeElapsed
0,5c1699b0c3658c25008e1192,city_chennai_v2,weekday,professional_bathroom_cleaning,Cleaning - Other,1,3022,2018,12,51,17,0,351,False,False,False,False,False,False,1545004808
1,5c1699b72dd2b92500f5f400,city_mumbai_v2,weekday,electricians,EPC,1,3303,2018,12,51,17,0,351,False,False,False,False,False,False,1545004815
2,5c1699df4fb96a2400166132,city_mumbai_v2,weekday,salon_at_home,Salon at Home,0,3330,2018,12,51,17,0,351,False,False,False,False,False,False,1545004855
3,5c1699f405cbce26002ff832,city_kolkata_v2,weekday,salon_at_home,Salon at Home,0,3330,2018,12,51,17,0,351,False,False,False,False,False,False,1545004876
4,5c1699fb02f88224006901f5,city_chennai_v2,weekday,plumbers,EPC,1,3303,2018,12,51,17,0,351,False,False,False,False,False,False,1545004883


We can specify the order to use for categorical variables if we wish:

In [43]:
display(df_raw.isnull().sum().sort_index()/len(df_raw))

category_key                  0.0
city_key                      0.0
event_timeDay                 0.0
event_timeDayofweek           0.0
event_timeDayofyear           0.0
event_timeElapsed             0.0
event_timeIs_month_end        0.0
event_timeIs_month_start      0.0
event_timeIs_quarter_end      0.0
event_timeIs_quarter_start    0.0
event_timeIs_year_end         0.0
event_timeIs_year_start       0.0
event_timeMonth               0.0
event_timeWeek                0.0
event_timeYear                0.0
record_id                     0.0
req_id                        0.0
rptcatg                       0.0
session_group                 0.0
weekday                       0.0
dtype: float64

In [0]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/uc-data')

In [0]:
df_raw = pd.read_feather('tmp/uc-data')

In [46]:
df_raw.head()

,record_id,city_key,weekday,category_key,rptcatg,req_id,session_group,event_timeYear,event_timeMonth,event_timeWeek,event_timeDay,event_timeDayofweek,event_timeDayofyear,event_timeIs_month_end,event_timeIs_month_start,event_timeIs_quarter_end,event_timeIs_quarter_start,event_timeIs_year_end,event_timeIs_year_start,event_timeElapsed
0,5c1699b0c3658c25008e1192,city_chennai_v2,weekday,professional_bathroom_cleaning,Cleaning - Other,1,3022,2018,12,51,17,0,351,False,False,False,False,False,False,1545004808
1,5c1699b72dd2b92500f5f400,city_mumbai_v2,weekday,electricians,EPC,1,3303,2018,12,51,17,0,351,False,False,False,False,False,False,1545004815
2,5c1699df4fb96a2400166132,city_mumbai_v2,weekday,salon_at_home,Salon at Home,0,3330,2018,12,51,17,0,351,False,False,False,False,False,False,1545004855
3,5c1699f405cbce26002ff832,city_kolkata_v2,weekday,salon_at_home,Salon at Home,0,3330,2018,12,51,17,0,351,False,False,False,False,False,False,1545004876
4,5c1699fb02f88224006901f5,city_chennai_v2,weekday,plumbers,EPC,1,3303,2018,12,51,17,0,351,False,False,False,False,False,False,1545004883


In [0]:
df, y, nas = proc_df(df_raw, 'req_id')

In [50]:
df.head()

,record_id,city_key,weekday,category_key,rptcatg,session_group,event_timeYear,event_timeMonth,event_timeWeek,event_timeDay,event_timeDayofweek,event_timeDayofyear,event_timeIs_month_end,event_timeIs_month_start,event_timeIs_quarter_end,event_timeIs_quarter_start,event_timeIs_year_end,event_timeIs_year_start,event_timeElapsed
0,3,4,4,10,2,3022,2018,12,51,17,0,351,False,False,False,False,False,False,1545004808
1,6,9,4,4,3,3303,2018,12,51,17,0,351,False,False,False,False,False,False,1545004815
2,12,9,4,17,8,3330,2018,12,51,17,0,351,False,False,False,False,False,False,1545004855
3,17,8,4,17,8,3330,2018,12,51,17,0,351,False,False,False,False,False,False,1545004876
4,19,4,4,9,3,3303,2018,12,51,17,0,351,False,False,False,False,False,False,1545004883


In [0]:
def split_vals(a,n): return a[:n], a[n:]
n_valid = 351107
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
raw_train, raw_valid = split_vals(df_raw, n_trn)

In [10]:
df_raw.head()

,record_id,city_key,event_time,weekday,category_key,rptcatg,req_id,session_group
0,5c1699b0c3658c25008e1192,city_chennai_v2,2018-12-17 00:00:08,weekday,professional_bathroom_cleaning,Cleaning - Other,5c1699d714bd522300fae954,3022.0
1,5c1699b72dd2b92500f5f400,city_mumbai_v2,2018-12-17 00:00:15,weekday,electricians,EPC,5c1699ed70fa4f2500d66e7c,3303.0
2,5c1699df4fb96a2400166132,city_mumbai_v2,2018-12-17 00:00:55,weekday,salon_at_home,Salon at Home,NaN,3330.0
3,5c1699f405cbce26002ff832,city_kolkata_v2,2018-12-17 00:01:16,weekday,salon_at_home,Salon at Home,NaN,3330.0
4,5c1699fb02f88224006901f5,city_chennai_v2,2018-12-17 00:01:23,weekday,plumbers,EPC,5c169a6c8c954c2200693538,3303.0


In [0]:
add_datepart(df_raw, 'event_time')

In [12]:
df_raw.head()

,record_id,city_key,weekday,category_key,rptcatg,req_id,session_group,event_timeYear,event_timeMonth,event_timeWeek,event_timeDay,event_timeDayofweek,event_timeDayofyear,event_timeIs_month_end,event_timeIs_month_start,event_timeIs_quarter_end,event_timeIs_quarter_start,event_timeIs_year_end,event_timeIs_year_start,event_timeElapsed
0,5c1699b0c3658c25008e1192,city_chennai_v2,weekday,professional_bathroom_cleaning,Cleaning - Other,5c1699d714bd522300fae954,3022.0,2018,12,51,17,0,351,False,False,False,False,False,False,1545004808
1,5c1699b72dd2b92500f5f400,city_mumbai_v2,weekday,electricians,EPC,5c1699ed70fa4f2500d66e7c,3303.0,2018,12,51,17,0,351,False,False,False,False,False,False,1545004815
2,5c1699df4fb96a2400166132,city_mumbai_v2,weekday,salon_at_home,Salon at Home,NaN,3330.0,2018,12,51,17,0,351,False,False,False,False,False,False,1545004855
3,5c1699f405cbce26002ff832,city_kolkata_v2,weekday,salon_at_home,Salon at Home,NaN,3330.0,2018,12,51,17,0,351,False,False,False,False,False,False,1545004876
4,5c1699fb02f88224006901f5,city_chennai_v2,weekday,plumbers,EPC,5c169a6c8c954c2200693538,3303.0,2018,12,51,17,0,351,False,False,False,False,False,False,1545004883


In [13]:
train_cats(df_raw)
df_raw.head()

,record_id,city_key,weekday,category_key,rptcatg,req_id,session_group,event_timeYear,event_timeMonth,event_timeWeek,event_timeDay,event_timeDayofweek,event_timeDayofyear,event_timeIs_month_end,event_timeIs_month_start,event_timeIs_quarter_end,event_timeIs_quarter_start,event_timeIs_year_end,event_timeIs_year_start,event_timeElapsed
0,5c1699b0c3658c25008e1192,city_chennai_v2,weekday,professional_bathroom_cleaning,Cleaning - Other,5c1699d714bd522300fae954,3022.0,2018,12,51,17,0,351,False,False,False,False,False,False,1545004808
1,5c1699b72dd2b92500f5f400,city_mumbai_v2,weekday,electricians,EPC,5c1699ed70fa4f2500d66e7c,3303.0,2018,12,51,17,0,351,False,False,False,False,False,False,1545004815
2,5c1699df4fb96a2400166132,city_mumbai_v2,weekday,salon_at_home,Salon at Home,NaN,3330.0,2018,12,51,17,0,351,False,False,False,False,False,False,1545004855
3,5c1699f405cbce26002ff832,city_kolkata_v2,weekday,salon_at_home,Salon at Home,NaN,3330.0,2018,12,51,17,0,351,False,False,False,False,False,False,1545004876
4,5c1699fb02f88224006901f5,city_chennai_v2,weekday,plumbers,EPC,5c169a6c8c954c2200693538,3303.0,2018,12,51,17,0,351,False,False,False,False,False,False,1545004883
